In [1]:
import pandas as pd
import os
import pysubgroup as ps
import numpy as np
from bcb import sgs

In [2]:
# Junta todos os arquivos da série histórica que estão dentro de uma mesma pasta
path = "Dados/"
all_entries = os.listdir(path)
file_names = [entry for entry in all_entries if os.path.isfile(os.path.join(path, entry))]

itbi = pd.DataFrame({})
for file_name in file_names:
    temp_data = pd.read_csv(path+file_name, delimiter=";", low_memory=False)
    itbi = pd.concat([itbi, temp_data], axis=0)

itbi['Data Quitacao Transacao Formatada'] = pd.to_datetime(itbi['Data Quitacao Transacao'], format='%d/%m/%Y')
itbi['Ano Avaliacao'] = itbi['Data Quitacao Transacao Formatada'].dt.year
itbi['Mes Avaliacao'] = itbi['Data Quitacao Transacao Formatada'].dt.month
itbi.sort_values(by='Data Quitacao Transacao Formatada', ascending=True, inplace=True)

In [3]:
itbi = itbi.rename(columns={'Area Construida Adquirida': 'area',
                            'Padrao Acabamento Unidade': 'padrao',
                            'Valor Base Calculo': 'preco'})

# Padronização dos nomes
itbi['Bairro'] = itbi['Bairro'].str.strip().str.upper()
itbi['Descricao Tipo Ocupacao Unidade'] = itbi['Descricao Tipo Ocupacao Unidade'].str.strip().str.upper()

itbi.head()

,Endereco,Bairro,Ano de Construcao Unidade,Area Terreno Total,area,Area Adquirida Unidades Somadas,padrao,Fracao Ideal Adquirida,Tipo Construtivo Preponderante,Descricao Tipo Ocupacao Unidade,Valor Declarado,preco,Zona Uso ITBI,Data Quitacao Transacao,Data Quitacao Transacao Formatada,Ano Avaliacao,Mes Avaliacao
81,RUA SANTA CATARINA 1466 - APT 2601 - LOURDES -...,LOURDES,2007,"2.475,00","344,09","344,09",P5,"0,020319",AP,RESIDENCIAL,"875.000,00","879.448,67",ZCBH,02/01/2008,2008-01-02,2008,1
77,RUA PROFESSOR MIGUEL DE SOUZA 113 - APT 102 - ...,BURITIS,1996,450,"136,8","136,8",P3,"0,166667",AP,RESIDENCIAL,"90.000,00","97.904,43",ZAP,02/01/2008,2008-01-02,2008,1
76,RUA PONTE NOVA 624 - APT 401 - COLEGIO BATISTA...,COLEGIO BATISTA,2006,411,"220,15","220,15",P4,"0,20655",AP,RESIDENCIAL,"240.000,00","268.636,27",ZAP,02/01/2008,2008-01-02,2008,1
75,RUA PAULO KRUGER MOURAO 230 - BLOCO 7 APT 302 ...,JARDIM GUANABARA,2001,"10.199,00","49,14","49,14",P2,"0,003186",AP,RESIDENCIAL,"52.500,00","52.500,00",ZAR2,02/01/2008,2008-01-02,2008,1
74,RUA PATRICIO BARBOSA 783 - BLOCO A6 APT 302 - ...,CONJUNTO CALIFORNIA II,1982,"41.830,00",75,75,P2,"0,001544",AP,RESIDENCIAL,"25.850,00","28.461,47",ZAP,02/01/2008,2008-01-02,2008,1


## Pré-processamento dos dados

In [4]:
# Cria as novas variáveis e filtra os dados

itbi['preco'] = itbi['preco'].str.replace(".", "", regex=False)
itbi['preco'] = itbi['preco'].str.replace(",", ".")
itbi['preco'] = itbi['preco'].astype(float)

itbi['area'] = itbi['area'].str.replace(".", "", regex=False)
itbi['area'] = itbi['area'].str.replace(",", ".")
itbi['area'] = itbi['area'].astype(float)

itbi = itbi.query("area >= 50")
itbi = itbi.query("preco >= 100000")
itbi = itbi.query("`Descricao Tipo Ocupacao Unidade` == 'RESIDENCIAL'")

itbi['idade'] = itbi['Ano Avaliacao'] - itbi['Ano de Construcao Unidade']
itbi['valor_m2'] = itbi['preco'] / itbi['area']

itbi = itbi.query("idade >= 0 and idade <= 100")

In [5]:
# Vamos trazer os preços dos imóveis para valor presente, pois precisamos comparar seus valores em anos diferentes

# Busca os dados do IGP-M pela biblioteca do banco central e cria o deflator
print("Buscando dados do IGP-M no Banco Central...")
igpm = sgs.get({'IGP-M': 189}, start='2008-01-01').rename(columns={'IGP-M': 'igpm_mensal'})
igpm['igpm_mensal'] = igpm['igpm_mensal'] / 100

data_referencia = igpm.index.max()
print(f"Todos os valores serão corrigidos para a data de referência: {data_referencia.strftime('%B de %Y')}")

valor_ref_acumulado = (1 + igpm['igpm_mensal']).cumprod().loc[data_referencia]
igpm['fator_acumulado'] = (1 + igpm['igpm_mensal']).cumprod()
igpm['fator_correcao'] = valor_ref_acumulado / igpm['fator_acumulado']
igpm['mes_ano'] = igpm.index.to_period('M')

# Aplicação do deflator pelo mês e ano de avaliação
itbi['mes_ano'] = itbi['Data Quitacao Transacao Formatada'].dt.to_period('M')

# Junta o fator de correção ao seu DataFrame principal
print("Juntando o fator de correção aos dados dos imóveis...")
itbi = pd.merge(itbi, igpm[['mes_ano', 'fator_correcao']], on='mes_ano', how='left')


# Aplica a correção e criar as novas colunas
print("Calculando os valores corrigidos...")
itbi['preco_corrigido'] = itbi['preco'] * itbi['fator_correcao']
itbi['valor_m2_corrigido'] = itbi['preco_corrigido'] / itbi['area']

print("\nCorreção inflacionária aplicada com sucesso usando o mês exato!")
itbi[["Ano Avaliacao", "Mes Avaliacao", 'preco', 'preco_corrigido', 'valor_m2', 'valor_m2_corrigido']].head()

Buscando dados do IGP-M no Banco Central...
Todos os valores serão corrigidos para a data de referência: June de 2025
Juntando o fator de correção aos dados dos imóveis...
Calculando os valores corrigidos...

Correção inflacionária aplicada com sucesso usando o mês exato!


,Ano Avaliacao,Mes Avaliacao,preco,preco_corrigido,valor_m2,valor_m2_corrigido
0,2008,1,879448.67,2.754822e+06,2555.868145,8006.108211
1,2008,1,268636.27,8.414875e+05,1220.241971,3822.336957
2,2008,1,361120.91,1.131190e+06,2119.005457,6637.661267
3,2008,1,365503.88,1.144920e+06,1261.532737,3951.677878
4,2008,1,311473.00,9.756710e+05,2166.617974,6786.804706


In [6]:
# Vamos realizar mais um refinamento nos dados: iremos eliminar os dados categóricos que aparecem poucas vezes
print(f"Tamanho original do dataset: {len(itbi):,}")

# Passo 1: Filtrar pelos tipos de imóvel de interesse
tipos_de_interesse = ['AP', 'CA']
itbi = itbi[itbi['Tipo Construtivo Preponderante'].isin(tipos_de_interesse)].copy()

print(f"Tamanho após filtrar por tipo de imóvel: {len(itbi):,}")

# Passo 2: Filtrar bairros com poucos imóveis
# Primeiro, contamos quantos imóveis cada bairro tem
contagem_bairros = itbi['Bairro'].value_counts()

# Definimos um limite mínimo. Podemos experimentar outros valores
limite_minimo_imoveis = 10 

# Pegamos a lista de bairros que ATENDEM ao critério
bairros_validos = contagem_bairros[contagem_bairros >= limite_minimo_imoveis].index

# Filtramos o DataFrame para manter apenas esses bairros
itbi = itbi[itbi['Bairro'].isin(bairros_validos)].copy()

print(f"Tamanho após remover bairros com menos de {limite_minimo_imoveis} imóveis: {len(itbi):,}")

Tamanho original do dataset: 312,468
Tamanho após filtrar por tipo de imóvel: 310,130
Tamanho após remover bairros com menos de 10 imóveis: 310,006


In [7]:
print(itbi.shape)
itbi.describe()

(310006, 23)


,Ano de Construcao Unidade,area,preco,Data Quitacao Transacao Formatada,Ano Avaliacao,Mes Avaliacao,idade,valor_m2,fator_correcao,preco_corrigido,valor_m2_corrigido
count,310006.000000,310006.000000,3.100060e+05,310006,310006.000000,310006.000000,310006.000000,310006.000000,310006.000000,3.100060e+05,310006.000000
mean,2000.395450,140.600373,4.623389e+05,2016-12-03 00:12:30.550634496,2016.409579,6.686345,16.014129,3275.190822,1.895691,8.112423e+05,5688.292313
min,1910.000000,50.000000,1.000000e+05,2008-01-02 00:00:00,2008.000000,1.000000,0.000000,22.713571,1.007347,1.010952e+05,41.947270
25%,1990.000000,79.000000,2.200000e+05,2012-07-06 00:00:00,2012.000000,4.000000,2.000000,2373.241336,1.121482,3.938624e+05,4109.908594
50%,2005.000000,118.000000,3.350000e+05,2016-12-09 00:00:00,2016.000000,7.000000,10.000000,3157.693236,1.846793,5.913993e+05,5436.277368
75%,2013.000000,169.000000,5.350000e+05,2021-06-09 00:00:00,2021.000000,10.000000,27.000000,3922.588316,2.438930,9.383352e+05,6807.820836
max,2024.000000,10189.340000,2.696200e+07,2024-12-31 00:00:00,2024.000000,12.000000,98.000000,76550.212088,3.185642,4.630771e+07,187283.688939
std,16.519796,122.972651,4.607199e+05,NaN,4.939305,3.365309,16.433323,1461.598438,0.663360,7.939042e+05,2369.539065


In [8]:
# Iremos utilizar o ano de avaliação como variável categórica para termos acesso mais fácil à série histórica
print(f"Tipo original da coluna 'Ano Avaliacao': {itbi['Ano Avaliacao'].dtype}")

# CONVERTER O ANO PARA UM TIPO CATEGÓRICO (string é a forma mais simples)
itbi['Ano Avaliacao'] = itbi['Ano Avaliacao'].astype(str)

print(f"Novo tipo da coluna 'Ano Avaliacao': {itbi['Ano Avaliacao'].dtype}")

Tipo original da coluna 'Ano Avaliacao': int32
Novo tipo da coluna 'Ano Avaliacao': object


## Teste 0: padrões apenas com os bairros

In [73]:
target = ps.NumericTarget('valor_m2')

include = ['Bairro']
ignore = list(set(list(itbi)) - set(include))
search_space = ps.create_selectors(itbi, ignore=ignore)

# Usaremos a StandardQF, que mede a diferença da média do subgrupo em desvios padrão
# O parâmetro 'a' ajuda a ponderar o tamanho do subgrupo.
quality_function = ps.StandardQFNumeric(a=0.5)

In [74]:
# Criando a tarefa de descoberta de subgrupos
task = ps.SubgroupDiscoveryTask(
    itbi,
    target,
    search_space,
    result_set_size=20,  # Queremos os 10 melhores subgrupos
    depth=1,             # Profundidade máxima da descrição (ex: Bairro='SAVASSI' AND Tipo_Imovel='APARTAMENTO')
    qf=quality_function
)

# Executando a busca
result = ps.BeamSearch().execute(task)

In [75]:
media_global_nominal = itbi['valor_m2'].mean()
media_global_real = itbi['valor_m2_corrigido'].mean()
df_results = result.to_dataframe()
medias_reais_sg = []

for index, row in df_results.iterrows():
    subgrupo_obj = row['subgroup']
    dados_do_subgrupo = subgrupo_obj.covers(itbi)
    media_real = itbi[dados_do_subgrupo]['valor_m2_corrigido'].mean()
    medias_reais_sg.append(media_real)
df_results['mean_sg_corrigido'] = medias_reais_sg

print("="*60)
print("Análise Detalhada dos Subgrupos de Maior Impacto Nominal")
print("="*60)
print(f"Média Global Nominal: R$ {media_global_nominal:,.2f}")
print(f"Média Global Real (corrigida): R$ {media_global_real:,.2f}\n")


for index, row in df_results.iterrows():
    # Coletando os valores da linha
    qualidade = row['quality']
    descricao = row['subgroup']
    tamanho_subgrupo = row['size_sg']
    media_nominal_sg = row['mean_sg']
    media_real_sg = row['mean_sg_corrigido']

    # Calculando os dois impactos percentuais
    impacto_nominal = ((media_nominal_sg / media_global_nominal) - 1) * 100
    impacto_real = ((media_real_sg / media_global_real) - 1) * 100

    # Imprimindo o "card" de resultado para cada subgrupo
    print(f"Qualidade: {qualidade:.3f} | Subgrupo: {descricao}")
    print(f"   > Tamanho: {tamanho_subgrupo:,.0f} imóveis")
    print(f"   > Média Nominal: R$ {media_nominal_sg:,.2f}  (Impacto: {impacto_nominal:+.2f}%)")
    print(f"   > Média Real   : R$ {media_real_sg:,.2f}  (Impacto: {impacto_real:+.2f}%)")
    print("-" * 40)

Análise Detalhada dos Subgrupos de Maior Impacto Nominal
Média Global Nominal: R$ 3,275.19
Média Global Real (corrigida): R$ 5,688.29

Qualidade: 145644.171 | Subgrupo: Bairro=='SAVASSI'
   > Tamanho: 5,992 imóveis
   > Média Nominal: R$ 5,156.70  (Impacto: +57.45%)
   > Média Real   : R$ 8,735.09  (Impacto: +53.56%)
----------------------------------------
Qualidade: 117348.980 | Subgrupo: Bairro=='SANTO AGOSTINHO'
   > Tamanho: 4,435 imóveis
   > Média Nominal: R$ 5,037.30  (Impacto: +53.80%)
   > Média Real   : R$ 8,948.41  (Impacto: +57.31%)
----------------------------------------
Qualidade: 112752.512 | Subgrupo: Bairro=='LOURDES'
   > Tamanho: 6,666 imóveis
   > Média Nominal: R$ 4,656.19  (Impacto: +42.17%)
   > Média Real   : R$ 8,281.02  (Impacto: +45.58%)
----------------------------------------
Qualidade: 84037.226 | Subgrupo: Bairro=='FUNCIONARIOS'
   > Tamanho: 3,071 imóveis
   > Média Nominal: R$ 4,791.65  (Impacto: +46.30%)
   > Média Real   : R$ 8,498.94  (Impacto: +49

## Primeiro Experimento: Comparação do m² com a Cidade Inteira

In [76]:
target = ps.NumericTarget('valor_m2')

include = ['Bairro', 'Tipo Construtivo Preponderante', 'Ano Avaliacao', 'idade']
ignore = list(set(list(itbi)) - set(include))
search_space = ps.create_selectors(itbi, ignore=ignore)

# Usaremos a StandardQF, que mede a diferença da média do subgrupo em desvios padrão
# O parâmetro 'a' ajuda a ponderar o tamanho do subgrupo.
quality_function = ps.StandardQFNumeric(a=0.5)

In [ ]:
# Criando a tarefa de descoberta de subgrupos
task = ps.SubgroupDiscoveryTask(
    itbi,
    target,
    search_space,
    result_set_size=10,  # 10 melhores subgrupos
    depth=3,             # Profundidade máxima da descrição 
    qf=quality_function
)

# Executando a busca
result = ps.BeamSearch().execute(task)

In [78]:
media_global_nominal = itbi['valor_m2'].mean()
media_global_real = itbi['valor_m2_corrigido'].mean()
df_results = result.to_dataframe()
medias_reais_sg = []

for index, row in df_results.iterrows():
    subgrupo_obj = row['subgroup']
    dados_do_subgrupo = subgrupo_obj.covers(itbi)
    media_real = itbi[dados_do_subgrupo]['valor_m2_corrigido'].mean()
    medias_reais_sg.append(media_real)
df_results['mean_sg_corrigido'] = medias_reais_sg

print("="*60)
print("Análise Detalhada dos Subgrupos de Maior Impacto Nominal")
print("="*60)
print(f"Média Global Nominal: R$ {media_global_nominal:,.2f}")
print(f"Média Global Real (corrigida): R$ {media_global_real:,.2f}\n")


for index, row in df_results.iterrows():
    # Coletando os valores da linha
    qualidade = row['quality']
    descricao = row['subgroup']
    tamanho_subgrupo = row['size_sg']
    media_nominal_sg = row['mean_sg']
    media_real_sg = row['mean_sg_corrigido']

    # Calculando os dois impactos percentuais
    impacto_nominal = ((media_nominal_sg / media_global_nominal) - 1) * 100
    impacto_real = ((media_real_sg / media_global_real) - 1) * 100

    # Imprimindo o "card" de resultado para cada subgrupo
    print(f"Qualidade: {qualidade:.3f} | Subgrupo: {descricao}")
    print(f"   > Tamanho: {tamanho_subgrupo:,.0f} imóveis")
    print(f"   > Média Nominal: R$ {media_nominal_sg:,.2f}  (Impacto: {impacto_nominal:+.2f}%)")
    print(f"   > Média Real   : R$ {media_real_sg:,.2f}  (Impacto: {impacto_real:+.2f}%)")
    print("-" * 40)

Análise Detalhada dos Subgrupos de Maior Impacto Nominal
Média Global Nominal: R$ 3,275.19
Média Global Real (corrigida): R$ 5,688.29

Qualidade: 188762.214 | Subgrupo: Ano Avaliacao=='2024' AND Tipo Construtivo Preponderante=='AP'
   > Tamanho: 18,845 imóveis
   > Média Nominal: R$ 4,650.24  (Impacto: +41.98%)
   > Média Real   : R$ 4,898.04  (Impacto: -13.89%)
----------------------------------------
Qualidade: 185965.314 | Subgrupo: Ano Avaliacao=='2024'
   > Tamanho: 21,522 imóveis
   > Média Nominal: R$ 4,542.82  (Impacto: +38.70%)
   > Média Real   : R$ 4,784.52  (Impacto: -15.89%)
----------------------------------------
Qualidade: 145644.171 | Subgrupo: Bairro=='SAVASSI'
   > Tamanho: 5,992 imóveis
   > Média Nominal: R$ 5,156.70  (Impacto: +57.45%)
   > Média Real   : R$ 8,735.09  (Impacto: +53.56%)
----------------------------------------
Qualidade: 140687.785 | Subgrupo: Bairro=='SAVASSI' AND Tipo Construtivo Preponderante=='AP'
   > Tamanho: 5,939 imóveis
   > Média Nominal

## Segundo Experimento: Comparação Local do m² dos Bairros

In [79]:
# Vamos pegar os 100 bairros com o maior número de imóveis na base
bairros_para_analise = itbi['Bairro'].value_counts().nlargest(100).index

print("Analisando as dinâmicas internas dos seguintes bairros:")
print(bairros_para_analise.tolist())
print("="*60)

Analisando as dinâmicas internas dos seguintes bairros:
['BURITIS', 'CASTELO', 'SAGRADA FAMILIA', 'SANTO ANTONIO', 'LOURDES', 'SION', 'OURO PRETO', 'SAVASSI', 'SERRA', 'SANTA AMELIA', 'PADRE EUSTAQUIO', 'GUTIERREZ', 'NOVA SUISSA', 'SANTO AGOSTINHO', 'PLANALTO', 'PRADO', 'SANTA EFIGENIA', 'ANCHIETA', 'CENTRO', 'SANTA MONICA', 'CIDADE NOVA', 'PAQUETA', 'FUNCIONARIOS', 'HAVAI', 'BELVEDERE', 'SAO JOAO BATISTA', 'ITAPOA', 'UNIAO', 'DONA CLARA', 'DIAMANTE', 'MANACAS', 'FERNAO DIAS', 'SANTA TEREZA', 'FLORESTA', 'LUXEMBURGO', 'ESTORIL', 'CAICARAS', 'IPIRANGA', 'SALGADO FILHO', 'CAMARGOS', 'SERRANO', 'NOVA GRANADA', 'JARDIM AMERICA', 'SAO PEDRO', 'PALMARES', 'CARLOS PRATES', 'SILVEIRA', 'CRUZEIRO', 'GRAJAU', 'COLEGIO BATISTA', 'CAICARA ADELAIDE', 'COPACABANA', 'CINQUENTENARIO', 'JARDIM GUANABARA', 'CORACAO EUCARISTICO', 'SANTA CRUZ', 'PALMEIRAS', 'HELIOPOLIS', 'SANTA INES', 'LIBERDADE', 'BARREIRO', 'SANTA BRANCA', 'SANTA ROSA', 'CORACAO DE JESUS', 'BARRO PRETO', 'SANTA TEREZINHA', 'ARAGUAIA', '

In [81]:
print("Iniciando Análise Focada por Bairro com comparação Nominal e Real...")
print("="*70)

for bairro_foco in bairros_para_analise:
    print(f"\n--- Análise Focada no Bairro: {bairro_foco} ---")
    
    # 1. Filtra o DataFrame para o bairro em questão
    df_bairro = itbi[itbi['Bairro'] == bairro_foco].copy()
    
    if len(df_bairro) < 50:
        print("Dados insuficientes para uma análise robusta. Pulando...")
        continue

    # 2. Calcula as médias de base PARA ESTE BAIRRO ESPECÍFICO
    media_bairro_nominal = df_bairro['valor_m2'].mean()
    media_bairro_real = df_bairro['valor_m2_corrigido'].mean()
    print(f"Média Nominal do Bairro: R$ {media_bairro_nominal:,.2f}")
    print(f"Média Real do Bairro: R$ {media_bairro_real:,.2f}\n")

    # 3. Executa a análise do pysubgroup
    target = ps.NumericTarget('valor_m2') # Alvo é o valor nominal
    quality_function = ps.StandardQFNumeric(a=0.5)
    
    include = ['Tipo Construtivo Preponderante', 'Ano Avaliacao', 'idade']
    ignore = list(set(list(itbi)) - set(include))
    search_space = ps.create_selectors(df_bairro, ignore=ignore)

    task = ps.SubgroupDiscoveryTask(
        df_bairro,
        target,
        search_space,
        result_set_size=5,
        depth=2,
        qf=quality_function
    )
    
    result = ps.BeamSearch().execute(task)
    df_results = result.to_dataframe()

    if df_results.empty:
        print("Nenhum padrão interessante encontrado para este bairro.")
        print("-" * 50)
        continue

    # --- INÍCIO DA ATUALIZAÇÃO NA IMPRESSÃO ---

    # 4. Imprime os resultados com as duas métricas de impacto
    for index, row in df_results.iterrows():
        subgrupo_obj = row['subgroup']
        
        # Filtra os dados do bairro para obter os dados apenas deste subgrupo
        dados_do_subgrupo = subgrupo_obj.covers(df_bairro)
        
        # Coleta as médias nominais e reais do subgrupo
        media_nominal_sg = row['mean_sg']
        media_real_sg = df_bairro[dados_do_subgrupo]['valor_m2_corrigido'].mean()
        
        # Calcula os dois impactos percentuais em relação às médias do BAIRRO
        impacto_nominal = ((media_nominal_sg / media_bairro_nominal) - 1) * 100
        impacto_real = ((media_real_sg / media_bairro_real) - 1) * 100
        
        # Imprime o "card" de resultado aprimorado
        print(f"Qualidade: {row['quality']:.2f} | Padrão Interno: {row['subgroup']}")
        print(f"   > Tamanho: {row['size_sg']:,.0f} imóveis")
        print(f"   > Média Nominal: R$ {media_nominal_sg:,.2f}  (Impacto: {impacto_nominal:+.2f}% vs. média NOMINAL do bairro)")
        print(f"   > Média Real   : R$ {media_real_sg:,.2f}  (Impacto: {impacto_real:+.2f}% vs. média REAL do bairro)")
        print("-" * 50)


Iniciando Análise Focada por Bairro com comparação Nominal e Real...

--- Análise Focada no Bairro: BURITIS ---
Média Nominal do Bairro: R$ 3,080.40
Média Real do Bairro: R$ 5,643.84

Qualidade: 48060.54 | Padrão Interno: Ano Avaliacao=='2024'
   > Tamanho: 984 imóveis
   > Média Nominal: R$ 4,612.52  (Impacto: +49.74% vs. média NOMINAL do bairro)
   > Média Real   : R$ 4,861.03  (Impacto: -13.87% vs. média REAL do bairro)
--------------------------------------------------
Qualidade: 46338.79 | Padrão Interno: Ano Avaliacao=='2024' AND Tipo Construtivo Preponderante=='AP'
   > Tamanho: 969 imóveis
   > Média Nominal: R$ 4,569.02  (Impacto: +48.33% vs. média NOMINAL do bairro)
   > Média Real   : R$ 4,816.28  (Impacto: -14.66% vs. média REAL do bairro)
--------------------------------------------------
Qualidade: 39855.07 | Padrão Interno: Ano Avaliacao=='2023'
   > Tamanho: 1,073 imóveis
   > Média Nominal: R$ 4,297.10  (Impacto: +39.50% vs. média NOMINAL do bairro)
   > Média Real   :

## Terceiro Experimento: Comparação Triangular dos Padrões

In [68]:
print("="*70)
print("Análise 1: Buscando segmentos por Padrão de Acabamento MÉDIO")
print("="*70)

# --- ETAPA 1: CRIAÇÃO DO ALVO ORDINAL ---
mapa_padrao = {'P1': 1, 'P2': 2, 'P3': 3, 'P4': 4, 'P5': 5}
itbi['padrao_ordinal'] = itbi['padrao'].map(mapa_padrao)

# --- ETAPA 2: CONFIGURAÇÃO DO PYSUBGROUP (NUMÉRICO) ---
target_ordinal = ps.NumericTarget('padrao_ordinal')
quality_function_numeric = ps.StandardQFNumeric(a=0.5)

include = ['Bairro', 'Ano Avaliacao', 'Tipo Construtivo Preponderante', 'idade']
ignore = list(set(list(itbi)) - set(include))
search_space = ps.create_selectors(itbi, ignore=ignore)

task_ordinal = ps.SubgroupDiscoveryTask(
    itbi, target_ordinal, search_space,
    result_set_size=15, depth=3, qf=quality_function_numeric
)

result_ordinal = ps.BeamSearch().execute(task_ordinal)
df_results_ordinal = result_ordinal.to_dataframe()

# --- ETAPA 3: IMPRESSÃO E INTERPRETAÇÃO (COM TAMANHO DO SUBGRUPO) ---
media_global_indice = itbi['padrao_ordinal'].mean()
print(f"Índice de Padrão Médio da Cidade: {media_global_indice:.2f}\n")

for index, row in df_results_ordinal.iterrows():
    print(f"Qualidade: {row['quality']:.3f} | Subgrupo: {row['subgroup']}")
    # --- LINHA ADICIONADA ---
    print(f"   > Tamanho do Subgrupo: {row['size_sg']:,.0f} imóveis")
    # --- FIM DA ADIÇÃO ---
    print(f"   > Índice de Padrão Médio do Subgrupo: {row['mean_sg']:.2f}")
    impacto = ((row['mean_sg'] / media_global_indice) - 1) * 100
    print(f"   > (Impacto: {impacto:+.2f}% vs. a média da cidade)")
    print("-" * 40)

Análise 1: Buscando segmentos por Padrão de Acabamento MÉDIO
Índice de Padrão Médio da Cidade: 3.22

Qualidade: 91.407 | Subgrupo: Tipo Construtivo Preponderante=='AP' AND idade: [1:6[
   > Tamanho do Subgrupo: 82,078 imóveis
   > Índice de Padrão Médio do Subgrupo: 3.54
   > (Impacto: +9.91% vs. a média da cidade)
----------------------------------------
Qualidade: 83.528 | Subgrupo: idade: [1:6[
   > Tamanho do Subgrupo: 87,597 imóveis
   > Índice de Padrão Médio do Subgrupo: 3.50
   > (Impacto: +8.77% vs. a média da cidade)
----------------------------------------
Qualidade: 71.931 | Subgrupo: Bairro=='BELVEDERE'
   > Tamanho do Subgrupo: 2,904 imóveis
   > Índice de Padrão Médio do Subgrupo: 4.55
   > (Impacto: +41.46% vs. a média da cidade)
----------------------------------------
Qualidade: 70.846 | Subgrupo: Bairro=='BELVEDERE' AND Tipo Construtivo Preponderante=='AP'
   > Tamanho do Subgrupo: 2,593 imóveis
   > Índice de Padrão Médio do Subgrupo: 4.61
   > (Impacto: +43.21% vs.

In [69]:
print("\n\n" + "="*70)
print("Análise 2: Buscando segmentos por Concentração de ALTO Padrão (P5)")
print("="*70)

# --- ETAPA 1: CRIAÇÃO DO ALVO BINÁRIO (P5) ---
itbi['eh_alto_padrao'] = (itbi['padrao'] == 'P5')

# --- ETAPA 2: CONFIGURAÇÃO DO PYSUBGROUP (BINÁRIO) ---
target_p5 = ps.BinaryTarget('eh_alto_padrao', True)
quality_function_binary = ps.WRAccQF()

task_p5 = ps.SubgroupDiscoveryTask(
    itbi, target_p5, search_space,
    result_set_size=15, depth=3, qf=quality_function_binary
)

result_p5 = ps.BeamSearch().execute(task_p5)
df_results_p5 = result_p5.to_dataframe()

# --- ETAPA 3: IMPRESSÃO E INTERPRETAÇÃO (COM CONTAGEM COMPLETA) ---
cobertura_global_p5 = itbi['eh_alto_padrao'].mean()
print(f"Concentração Global de P5: {cobertura_global_p5:.2%}\n")

for index, row in df_results_p5.iterrows():
    cobertura_subgrupo = row['positives_sg'] / row['size_sg']
    print(f"Qualidade (WRAcc): {row['quality']:.4f} | Subgrupo: {row['subgroup']}")
    # --- LINHA MODIFICADA ---
    print(f"   > Concentração de P5: {cobertura_subgrupo:.2%} ({row['positives_sg']:,.0f} de {row['size_sg']:,.0f} imóveis)")
    # --- FIM DA MODIFICAÇÃO ---
    print("-" * 40)



Análise 2: Buscando segmentos por Concentração de ALTO Padrão (P5)
Concentração Global de P5: 4.79%

Qualidade (WRAcc): 0.0090 | Subgrupo: Tipo Construtivo Preponderante=='AP' AND idade: [1:6[
   > Concentração de P5: 8.20% (6,734 de 82,078 imóveis)
----------------------------------------
Qualidade (WRAcc): 0.0085 | Subgrupo: idade: [1:6[
   > Concentração de P5: 7.79% (6,821 de 87,597 imóveis)
----------------------------------------
Qualidade (WRAcc): 0.0050 | Subgrupo: Bairro=='BELVEDERE'
   > Concentração de P5: 57.82% (1,679 de 2,904 imóveis)
----------------------------------------
Qualidade (WRAcc): 0.0047 | Subgrupo: Bairro=='SANTO AGOSTINHO'
   > Concentração de P5: 37.95% (1,683 de 4,435 imóveis)
----------------------------------------
Qualidade (WRAcc): 0.0047 | Subgrupo: Bairro=='SANTO AGOSTINHO' AND Tipo Construtivo Preponderante=='AP'
   > Concentração de P5: 38.39% (1,679 de 4,373 imóveis)
----------------------------------------
Qualidade (WRAcc): 0.0047 | Subgrupo:

In [70]:
print("\n\n" + "="*70)
print("Análise 3: Buscando segmentos por Concentração de Padrão BÁSICO (P1)")
print("="*70)

# --- ETAPA 1: CRIAÇÃO DO ALVO BINÁRIO (P1) ---
itbi['eh_padrao_basico'] = (itbi['padrao'] == 'P1')

# --- ETAPA 2: CONFIGURAÇÃO DO PYSUBGROUP (BINÁRIO) ---
target_p1 = ps.BinaryTarget('eh_padrao_basico', True)
quality_function_binary = ps.WRAccQF() 

task_p1 = ps.SubgroupDiscoveryTask(
    itbi, target_p1, search_space,
    result_set_size=15, depth=3, qf=quality_function_binary
)

result_p1 = ps.BeamSearch().execute(task_p1)
df_results_p1 = result_p1.to_dataframe()

# --- ETAPA 3: IMPRESSÃO E INTERPRETAÇÃO (COM CONTAGEM COMPLETA) ---
cobertura_global_p1 = itbi['eh_padrao_basico'].mean()
print(f"Concentração Global de P1: {cobertura_global_p1:.2%}\n")

for index, row in df_results_p1.iterrows():
    cobertura_subgrupo = row['positives_sg'] / row['size_sg']
    print(f"Qualidade (WRAcc): {row['quality']:.4f} | Subgrupo: {row['subgroup']}")
    # --- LINHA MODIFICADA ---
    print(f"   > Concentração de P1: {cobertura_subgrupo:.2%} ({row['positives_sg']:,.0f} de {row['size_sg']:,.0f} imóveis)")
    # --- FIM DA MODIFICAÇÃO ---
    print("-" * 40)



Análise 3: Buscando segmentos por Concentração de Padrão BÁSICO (P1)
Concentração Global de P1: 0.72%

Qualidade (WRAcc): 0.0044 | Subgrupo: Tipo Construtivo Preponderante=='CA'
   > Concentração de P1: 3.88% (1,660 de 42,768 imóveis)
----------------------------------------
Qualidade (WRAcc): 0.0024 | Subgrupo: Tipo Construtivo Preponderante=='CA' AND idade>=31
   > Concentração de P1: 4.55% (892 de 19,606 imóveis)
----------------------------------------
Qualidade (WRAcc): 0.0022 | Subgrupo: idade>=31
   > Concentração de P1: 1.79% (1,154 de 64,488 imóveis)
----------------------------------------
Qualidade (WRAcc): 0.0012 | Subgrupo: Tipo Construtivo Preponderante=='CA' AND idade: [16:31[
   > Concentração de P1: 4.51% (442 de 9,807 imóveis)
----------------------------------------
Qualidade (WRAcc): 0.0006 | Subgrupo: idade: [16:31[
   > Concentração de P1: 1.05% (625 de 59,783 imóveis)
----------------------------------------
Qualidade (WRAcc): 0.0006 | Subgrupo: Tipo Construtiv

## Quarto Experimento: Comparação Triangular por Bairro

In [72]:
# --- ETAPA 1: PREPARAÇÃO GLOBAL (executada apenas uma vez) ---

print("Preparando dados para a Análise Triangular Focada...")

# Define os padrões de interesse
PADRAO_LUXO = 'P5'
PADRAO_BASICO = 'P1'

# Cria as colunas alvo para as 3 análises
itbi['eh_alto_padrao'] = (itbi['padrao'] == PADRAO_LUXO)
itbi['eh_padrao_basico'] = (itbi['padrao'] == PADRAO_BASICO)
mapa_padrao = {'P1': 1, 'P2': 2, 'P3': 3, 'P4': 4, 'P5': 5}
itbi['padrao_ordinal'] = itbi['padrao'].map(mapa_padrao)

# Garante que o ano será tratado como categoria
itbi['Ano Avaliacao'] = itbi['Ano Avaliacao'].astype(str)

# Seleciona os bairros de interesse (ex: os 100 maiores)
bairros_para_analise = itbi['Bairro'].value_counts().nlargest(100).index

print(f"Analisando {len(bairros_para_analise)} bairros...")
print("="*80)


# --- ETAPA 2: LOOP PRINCIPAL DE ANÁLISE POR BAIRRO ---

for bairro_foco in bairros_para_analise:
    print(f"\n\n### RELATÓRIO DO BAIRRO: {bairro_foco} ###")
    
    # Filtra o DataFrame para o bairro em questão
    df_bairro = itbi[itbi['Bairro'] == bairro_foco].copy()
    
    if len(df_bairro) < 50:
        print("Dados insuficientes para uma análise robusta. Pulando...")
        print("-" * 80)
        continue

    # --- 2.1 Análise de Padrão MÉDIO (Ordinal/Numérico) ---
    print("\n--- 1. Análise de Padrão MÉDIO ---")
    media_bairro_indice = df_bairro['padrao_ordinal'].mean()
    print(f"Índice de Padrão Médio do Bairro: {media_bairro_indice:.2f}\n")

    include = ['Ano Avaliacao', 'Tipo Construtivo Preponderante', 'idade']
    ignore = list(set(list(itbi)) - set(include))
    task_ordinal = ps.SubgroupDiscoveryTask(
        df_bairro, ps.NumericTarget('padrao_ordinal'),
        ps.create_selectors(df_bairro, ignore=ignore),
        result_set_size=5, depth=2, qf=ps.StandardQFNumeric(a=0.5)
    )
    result_ordinal = ps.BeamSearch().execute(task_ordinal)
    for _, row in result_ordinal.to_dataframe().iterrows():
        print(f"Padrão: {row['subgroup']}")
        print(f"  > Tamanho: {row['size_sg']:,.0f} imóveis | Índice Médio: {row['mean_sg']:.2f}")

    # --- 2.2 Análise de Concentração de ALTO Padrão (P5) ---
    print("\n--- 2. Análise de Concentração de ALTO Padrão (P5) ---")
    total_p5_bairro = df_bairro['eh_alto_padrao'].sum()
    
    if total_p5_bairro > 0:
        cobertura_bairro_p5 = total_p5_bairro / len(df_bairro)
        print(f"Concentração de P5 no Bairro: {cobertura_bairro_p5:.2%} ({total_p5_bairro:,.0f} de {len(df_bairro):,.0f} imóveis)\n")
        
        task_p5 = ps.SubgroupDiscoveryTask(
            df_bairro, ps.BinaryTarget('eh_alto_padrao', True),
            ps.create_selectors(df_bairro, ignore=ignore),
            result_set_size=3, depth=2, qf=ps.WRAccQF()
        )
        result_p5 = ps.BeamSearch().execute(task_p5)
        for _, row in result_p5.to_dataframe().iterrows():
            print(f"Padrão: {row['subgroup']}")
            print(f"  > Concentração P5: {(row['positives_sg']/row['size_sg']):.2%} ({row['positives_sg']:,.0f} de {row['size_sg']:,.0f} imóveis)")
    else:
        print(f"O bairro não possui imóveis de padrão '{PADRAO_LUXO}' para análise.")

    # --- 2.3 Análise de Concentração de Padrão BÁSICO (P1) ---
    print("\n--- 3. Análise de Concentração de Padrão BÁSICO (P1) ---")
    total_p1_bairro = df_bairro['eh_padrao_basico'].sum()

    if total_p1_bairro > 0:
        cobertura_bairro_p1 = total_p1_bairro / len(df_bairro)
        print(f"Concentração de P1 no Bairro: {cobertura_bairro_p1:.2%} ({total_p1_bairro:,.0f} de {len(df_bairro):,.0f} imóveis)\n")
        
        task_p1 = ps.SubgroupDiscoveryTask(
            df_bairro, ps.BinaryTarget('eh_padrao_basico', True),
            ps.create_selectors(df_bairro, ignore=ignore),
            result_set_size=3, depth=2, qf=ps.WRAccQF()
        )
        result_p1 = ps.BeamSearch().execute(task_p1)
        for _, row in result_p1.to_dataframe().iterrows():
            print(f"Padrão: {row['subgroup']}")
            print(f"  > Concentração P1: {(row['positives_sg']/row['size_sg']):.2%} ({row['positives_sg']:,.0f} de {row['size_sg']:,.0f} imóveis)")
    else:
        print(f"O bairro não possui imóveis de padrão '{PADRAO_BASICO}' para análise.")
    
    print("-" * 80)

Preparando dados para a Análise Triangular Focada...
Analisando 100 bairros...


### RELATÓRIO DO BAIRRO: BURITIS ###

--- 1. Análise de Padrão MÉDIO ---
Índice de Padrão Médio do Bairro: 3.61

Padrão: idade: [1:4[
  > Tamanho: 6,032 imóveis | Índice Médio: 3.78
Padrão: Tipo Construtivo Preponderante=='AP' AND idade: [1:4[
  > Tamanho: 5,851 imóveis | Índice Médio: 3.77
Padrão: Ano Avaliacao=='2011' AND idade<1
  > Tamanho: 315 imóveis | Índice Médio: 4.06
Padrão: Tipo Construtivo Preponderante=='AP' AND idade<1
  > Tamanho: 2,358 imóveis | Índice Médio: 3.77
Padrão: idade<1
  > Tamanho: 2,377 imóveis | Índice Médio: 3.76

--- 2. Análise de Concentração de ALTO Padrão (P5) ---
Concentração de P5 no Bairro: 3.57% (784 de 21,934 imóveis)

Padrão: idade: [1:4[
  > Concentração P5: 5.11% (308 de 6,032 imóveis)
Padrão: Ano Avaliacao=='2011' AND idade<1
  > Concentração P5: 26.03% (82 de 315 imóveis)
Padrão: Tipo Construtivo Preponderante=='AP' AND idade<1
  > Concentração P5: 6.45% (152 de 